##  09 - Learning Policy on Attributes 
### Attributes from Claudia' simulator

IDEA:: 
Group vehicles based on their state. Represent states instead of vehicles and make net output location of the desired vehicle  NOT WORKING

RIGHT NOW: X is the same but truing to predict position of the vehicle to select

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import random
import tqdm

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch import optim

In [2]:
class Net(nn.Module):
    def __init__(self, inp, num_v):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(inp, 200)
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 200)
        self.bn2 = nn.BatchNorm1d(200)
        self.d2  = nn.Dropout(p=0.2)
        #self.fc3 = nn.Linear(500, 1000)
        #self.bn3 = nn.BatchNorm1d(1000)
        self.fc5 = nn.Linear(200, 200)
        self.bn5 = nn.BatchNorm1d(200)
        self.d5  = nn.Dropout(p=0.5)
        self.fc6 = nn.Linear(200, 2)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.d2(x)
        #x = F.relu(self.bn3(self.fc3(x)))
        x = F.relu(self.bn5(self.fc5(x)))
        #x = self.d5(x)
        x = self.fc6(x)
        #return F.log_softmax(x)
        return x

In [3]:
MAX_V = 30
mini_batch_size = 50
#n_epochs=150
n_epochs=25

tables = list(range(1,95))
random.shuffle(tables)


train_tables, test_tables, validation_tables = \
tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.9)], tables[int(len(tables)*0.9):]

In [5]:
lr = 0.001
inp_size = 4 + MAX_V*6

model = Net(inp_size, MAX_V)
optimizer = optim.Adam(model.parameters(), lr=lr)
#optimizer = optim.SGD(lr=lr)
criterion = nn.MSELoss()

In [6]:
def epoch_train(model, train_table, test_tables, optimizer, criterion, e, train=True):
    if train:
        TABLES = train_tables
        st = 'train'
    else:
        TABLES = test_tables
        st = 'test'
    
    
    sum_loss = 0
    acc = []
    idx_failures = []

    # train data
    for k, TABLE in enumerate(TABLES):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        #random.shuffle(idx)

        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.asarray([np.hstack(data[i]) for i in t_idx])
            x_toy = np.asarray([np.array(data[i]) for i in t_idx])
            veh = np.hstack([data_y[i] for i in t_idx])
            y = [x_toy[i][int(veh[i]) + 1][2:4] for i in range(x_toy.shape[0])] #position of each vehicle to be selected
            
            #x_set = []
            #for i in range(x.shape[0]):
                #x_state = [x[i][0][k] for k in range(len(x[i][0]))]
                #visited = []
                #for j in range(1, x.shape[1]):
                #    if x[i][j] not in visited:
                #        visited.append(x[i][j])
                #        for k in x[i][j]:
                #            x_state.append(k)
                #x_flat = np.zeros(4 + 6*num_v)
                #x_flat[:len(x_state)] = x_state

                #x_set.append(x_flat)
            #x_set = np.asarray(x_set)  
                
            train_x = torch.tensor(x).type(torch.FloatTensor)
            #train_x = torch.tensor(x_set).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output = model(train_x)
            batch_loss = criterion(output, train_y)
            
            if train:
                batch_loss.backward()
                optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
                
    return sum_loss

In [7]:
train_loss, acc, test_loss, test_acc, idx_f = [], [], [], [], []
for epoch in range(n_epochs):
    train_l  = epoch_train(model, train_tables, test_tables, optimizer, criterion, epoch)
    test_l  = epoch_train(model, train_tables, test_tables, optimizer, criterion, epoch, train=False)
    
    print('\r Epoch {}:\tTrain loss: {:.4f}\tTest loss: {:.4f}'.format(epoch, train_l, test_l))
    train_loss.append(train_l)
    test_loss.append(test_l)

 Epoch 0:	Train loss: 11.5997	Test loss: 4.4596
 Epoch 1:	Train loss: 7.4150	Test loss: 4.1722
 Epoch 2:	Train loss: 6.4396	Test loss: 4.1175
 Epoch 3:	Train loss: 5.9107	Test loss: 4.2358
 Epoch 4:	Train loss: 5.4575	Test loss: 4.3424
 Epoch 5:	Train loss: 5.2324	Test loss: 4.3370
 Epoch 6:	Train loss: 5.0300	Test loss: 4.3423
 Epoch 7:	Train loss: 4.7880	Test loss: 4.4669
 Epoch 8:	Train loss: 4.5250	Test loss: 4.4831
 Epoch 9:	Train loss: 4.3696	Test loss: 4.6900
 Epoch 10:	Train loss: 4.2317	Test loss: 4.3180
 Epoch 11:	Train loss: 4.1221	Test loss: 4.3380
 Epoch 12:	Train loss: 3.9372	Test loss: 4.4184
 Epoch 13:	Train loss: 3.8580	Test loss: 4.4183
 Epoch 14:	Train loss: 3.7141	Test loss: 4.7695
 Epoch 15:	Train loss: 3.6883	Test loss: 4.7370
 Epoch 16:	Train loss: 3.5507	Test loss: 4.5560
 Epoch 17:	Train loss: 3.4177	Test loss: 4.3471
 Epoch 18:	Train loss: 3.1426	Test loss: 4.4884
 Epoch 19:	Train loss: 3.2784	Test loss: 4.6744
 Epoch 20:	Train loss: 3.0782	Test loss: 4.6858
 

In [8]:
data = np.load('./minmax_data/data_vector_{}.npy'.format(2), allow_pickle=True).tolist()
data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(2), allow_pickle=True).tolist()

In [11]:
num_v = 30
idx = list(data.keys())
t_idx = idx[0:0+10]
x = np.asarray([np.array(data[i]) for i in t_idx])
veh = np.hstack([data_y[i] for i in t_idx])
y = [x[i][int(veh[i]) + 1][2:4] for i in range(x.shape[0])] #position of each vehicle to be selected
            
x_set = []
for i in range(x.shape[0]):
    x_state = [x[i][0][k] for k in range(len(x[i][0]))]
    visited = []
    for j in range(1, x.shape[1]):
        if x[i][j] not in visited:
            visited.append(x[i][j])
            for k in x[i][j]:
                x_state.append(k)
    x_flat = np.zeros(4 + 6*num_v)
    x_flat[:len(x_state)] = x_state
    
    x_set.append(x_flat)
x_set = np.asarray(x_set)  

In [12]:
X = torch.tensor(x_set).type(torch.FloatTensor)
o = model(X)

In [13]:
o

tensor([[ 0.2145, -0.0353],
        [ 0.5907,  0.3776],
        [ 0.5829,  0.3338],
        [ 0.6590,  0.5182],
        [ 0.7949,  0.5306],
        [ 0.5839,  0.6126],
        [ 0.6270,  0.6833],
        [ 0.3288,  0.4380],
        [ 0.4866,  0.3910],
        [ 0.6842,  0.4781]], grad_fn=<AddmmBackward>)

In [16]:
y

[[0.82, 0.69],
 [0.82, 0.69],
 [0.82, 0.69],
 [0.062, 0.193],
 [0.82, 0.69],
 [0.82, 0.69],
 [0.82, 0.69],
 [0.046, 0.169],
 [0.038, 0.107],
 [0.092, 0.096]]

In [14]:
a = np.asarray([np.asarray(x_state[i]) for i in range(len(x_state))])

In [15]:
a.flatten()

array([0.07740418, 0.19111497, 0.56977278, 0.18405474, 0.        ,
       1.        , 0.82      , 0.69      , 0.82      , 0.69      ,
       1.        , 1.        , 0.013     , 0.113     , 0.062     ,
       0.193     , 1.        , 1.        , 0.066     , 0.193     ,
       0.543     , 0.397     , 1.        , 1.        , 0.092     ,
       0.096     , 0.532     , 0.76      , 0.        , 0.        ,
       0.82      , 0.69      , 0.82      , 0.69      ])

In [37]:
idx = list(data.keys())
for b in range(0, len(idx), 10):
    t_idx = idx[b:b+10]
    x = np.asarray([np.array(data[i]) for i in t_idx])
    y = np.asarray([data_y[i] for i in t_idx])
    break

In [51]:
states = []
for i in range(x.shape[0]):
    states.append(x[i][int(y[i]) + 1][2:4]) # position of the chosen vehicle

In [52]:
states

[[0.895592925387986, 0.8729613295638093],
 [0.895592925387986, 0.8729613295638093],
 [0.895592925387986, 0.8729613295638093],
 [-1.960965869434946, -0.9093004769250232],
 [0.895592925387986, 0.8729613295638093],
 [0.895592925387986, 0.8729613295638093],
 [0.895592925387986, 0.8729613295638093],
 [-2.0243872323871726, -0.9963112925149629],
 [-2.0518770381718054, -1.21880203572071],
 [-1.8484091843313626, -1.2565651189936877]]

In [33]:
a = []
for i in range(x.shape[1]):
    if x[7][i] not in a:
        a.append(x[7][i])


In [34]:
a

[[-1.8484091843313626,
  -1.2565651189936877,
  -0.19057908585813527,
  1.1209518867351176],
 [0,
  1,
  0.895592925387986,
  0.8729613295638093,
  0.895592925387986,
  0.8729613295638093],
 [1,
  1,
  -2.1481995860003456,
  -1.1981685984672956,
  -1.960965869434946,
  -0.9093004769250232],
 [1,
  0,
  -2.0518770381718054,
  -1.21880203572071,
  -0.15572980450918536,
  -0.25136634565584237],
 [1,
  0,
  -2.0243872323871726,
  -0.9963112925149629,
  -0.19036263069502168,
  0.9611400755585784],
 [0,
  0,
  0.895592925387986,
  0.8729613295638093,
  0.895592925387986,
  0.8729613295638093]]